In [1]:
from robot_library_py import *
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Joy, JointState
from intera_core_msgs.msg import JointCommand

from threading import Thread
import time
import numpy as np 
from std_msgs.msg import Bool
import rotm2euler
 
import cv2
from matplotlib import pyplot as plt
import pickle
import datetime
import os 
import argparse

import argparse
import json
import numpy as np
import time
import os
import shutil
import psutil
import sys
import socket
import traceback

from collections import OrderedDict

import torch
from torch.utils.data import DataLoader

import robomimic
import robomimic.utils.train_utils as TrainUtils
import robomimic.utils.torch_utils as TorchUtils
import robomimic.utils.obs_utils as ObsUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.file_utils as FileUtils
from robomimic.config import config_factory
from robomimic.algo import algo_factory, RolloutPolicy
from robomimic.utils.log_utils import PrintLogger, DataLogger, flush_warnings

import matplotlib.pyplot as plt
import h5py

import os
import time 

from utils_ros import ROSInterface
import traceback

ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /home/carl/robomimic/robomimic/scripts/setup_macros.py
)


In [2]:
import beepy as beep
beep_start = lambda : beep.beep('coin')

In [3]:
tb=Thread(target=beep_start)
tb.start()

ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred


In [4]:
# ckpt_path = "/home/carl/data_sawyer/models/sawyer_spoonpick/model_epoch_600.pth" #good, upward
ckpt_path = "/home/carl/data_sawyer/models/sawyer_spoonpick/model_epoch_400.pth"   #good, pull, recorded
# ckpt_path = "/home/carl/data_sawyer/models/sawyer_spoonpick/model_epoch_500.pth"  #good, upward


#p20i
# ckpt_path = "/home/carl/data_sawyer/models/sawyer_spoonpick/p20i/model_epoch_600.pth"
# ckpt_path = "/home/carl/data_sawyer/models/sawyer_spoonpick/p20i/model_epoch_400.pth"   #best, with release also.

# ckpt_path = "/home/carl/data_sawyer/models/sawyer_spoonpick/g80b20/model_epoch_400.pth"  #new, g80b20,  10 failed inference
# ckpt_path = "/home/carl/data_sawyer/models/sawyer_spoonpick/g80b20/model_epoch_500.pth"  #new, g80b20, all failed, different way
# ckpt_path = "/home/carl/data_sawyer/models/sawyer_spoonpick/g80b20/model_epoch_600.pth"  #new, g80b20


#masked
ckpt_path = "/home/carl/data_sawyer/models/sawyer_spoonpick/masked_g80b20/model_epoch_600.pth"
assert os.path.exists(ckpt_path)

In [4]:
#14 march

# ckpt_path = "/home/carl/data_sawyer/models/sawyer_spoonpick/march14/p20b_model_epoch_400.pth"
# ckpt_path = "/home/carl/data_sawyer/models/sawyer_spoonpick/march14/p20tg_model_epoch_400.pth"
ckpt_path = "/home/carl/data_sawyer/models/sawyer_spoonpick/march14/p20o_model_epoch_400.pth"

In [5]:
device = TorchUtils.get_torch_device(try_to_use_cuda=True)

# restore policy
policy, ckpt_dict = FileUtils.policy_from_checkpoint(ckpt_path=ckpt_path, device=device, verbose=True)
pass 

/home/carl/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/carl/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/carl/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


============= Loaded Config =============
{
    "algo_name": "bc",
    "experiment": {
        "name": "sawyer_spoon",
        "validate": false,
        "logging": {
            "terminal_output_to_txt": true,
            "log_tb": true,
            "log_wandb": false,
            "wandb_proj_name": "spoon"
        },
        "save": {
            "enabled": true,
            "every_n_seconds": null,
            "every_n_epochs": 20,
            "epochs": [],
            "on_best_validation": false,
            "on_best_rollout_return": false,
            "on_best_rollout_success_rate": true
        },
        "epoch_every_n_steps": 500,
        "validation_epoch_every_n_steps": 50,
        "env": null,
        "additional_envs": null,
        "render": false,
        "render_video": true,
        "keep_all_videos": false,
        "video_skip": 5,
        "rollout": {
            "enabled": false,
            "n": 50,
            "horizon": 1000,
            "rate": 20,
            "w

In [6]:
ckpt_dict = FileUtils.maybe_dict_from_checkpoint(ckpt_path=ckpt_path, ckpt_dict=None)
# algo name and config from model dict
algo_name, _ = FileUtils.algo_name_from_checkpoint(ckpt_dict=ckpt_dict)
config, _ = FileUtils.config_from_checkpoint(algo_name=algo_name, ckpt_dict=ckpt_dict, verbose=False)

config.unlock()
config.train.data ="/home/carl/data_sawyer/spoon_pick/expert/spoon_pick_100_10d.hdf5"
config.lock()

ObsUtils.initialize_obs_utils_with_config(config)
dataset_path = os.path.expanduser(config.train.data)
shape_meta = FileUtils.get_shape_metadata_from_dataset(
    dataset_path=config.train.data,
    all_obs_keys=config.all_obs_keys,
    verbose=True
)


============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['robot0_eef_pos']
using obs modality: rgb with keys: ['robot0_eye_in_hand_image', 'agentview_image']
using obs modality: depth with keys: []
using obs modality: scan with keys: []
obs key agentview_image with shape (84, 84, 3)
obs key robot0_eef_pos with shape (6,)
obs key robot0_eye_in_hand_image with shape (84, 84, 3)


In [7]:
config.unlock()
config.train.data ="/home/carl/data_sawyer/spoon_pick/expert/spoon_pick_100_10d.hdf5"
config.lock()

In [8]:
def demo2actions(hdf5_file, demo_key, rollout_policy): 
    rollout_policy.start_episode()


    demo=hdf5_file['data'][demo_key]
    actions_real=demo['actions']
    obs=demo['obs']
    N=obs['robot0_eye_in_hand_image'].shape[0]

    obss=[]
    for i in range(N):
        ob={k:obs[k][i] for k in obs.keys()} 
        ob['robot0_eye_in_hand_image']=ob['robot0_eye_in_hand_image'].transpose(2,0,1)
        ob['agentview_image']=ob['agentview_image'].transpose(2,0,1)
        obss.append(ob)

    acs=[]
    for ob in obss: 
        ob = rollout_policy._prepare_observation(ob)
        ac = rollout_policy.policy.get_action(obs_dict=ob, goal_dict=None)
        ac=ac.detach().cpu().numpy()[0]
        acs.append(ac)

    actions_pred=np.array(acs)
    return actions_real, actions_pred

In [9]:
rclpy.init()

urdf_path = os.getcwd() +'/'+'sawyer.urdf'  #'/home/carl/sawyer_robot_ros2/src/teleop_script/sawyer.urdf'
robot = URDFModel(urdf_path)
jointMap = {name: ind for ind, name in enumerate(robot.jointNames)}


joint_names=['right_j0', 'right_j1', 'right_j2', 'right_j3', 'right_j4', 'right_j5', 'right_j6']
joint_inds=[jointMap[name] for name in joint_names]


node = Node('infer_live')
q = robot.getJoints()
ros_interface = ROSInterface(node, robot)
cmd_pub = node.create_publisher(JointCommand, '/robot/limb/right/joint_command', 10)

t1 = Thread(target=ros_interface.spin_thread)
t1.start()

sawyer
link: 0
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: base
link: 1
	cog: 0 0 0
	inertia: 1e-08 1e-08 1e-08     0     0     0
	mass: 0.0001
	name: torso
link: 2
	cog:    0    0 -0.5
	inertia:    5.06359    6.08689    4.96192 0.00105311   0.801996 0.00103417
	mass: 60.864
	name: pedestal
link: 3
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: controller_box
link: 4
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: pedestal_feet
link: 5
	cog:  -0.0006241 -2.8025e-05    0.065404
	inertia:   0.0067599   0.0067877   0.0074031  1.5888e-05 -6.1904e-07 -4.2024e-05
	mass: 2.0687
	name: right_arm_base_link
link: 6
	cog: 0.024366 0.010969  0.14363
	inertia:  0.053314  0.057902  0.023659 0.0080179  0.011734 0.0047093
	mass: 5.3213
	name: right_l0
link: 7
	cog:   0.0053207 -2.6549e-05      0.1021
	inertia:    0.011833   0.0082709   0.0049661  4.2124e-07  4.9425e-05 -4.4669e-06
	mass: 1.5795
	name: head
link: 8
	cog: 0 0 0
	inertia: 1e-08 1e-08 1e-08     0     0     0
	mass: 0.0001
	name

In [10]:
def get_ee(jointmsg):
    msg=jointmsg    
    q =  robot.getJoints() 
    for ind, name in enumerate(msg.name):
        if name in jointMap:
            q[jointMap[name]] = msg.position[ind]
    robot.setJoints(q)
    
    # T=robot.getBodyTransform('right_l6');
    T=robot.getBodyTransform('camera_link');
    xyz=T[0:3,3]
    e1 = rotm2euler.rotationMatrixToEulerAngles(T[0:3, 0:3])
    ee6=np.concatenate([xyz, e1])
    return ee6

def set_vels(vels):
    cmd_msg = JointCommand()
    cmd_msg.names = robot.jointNames
    cmd_msg.mode = cmd_msg.VELOCITY_MODE
    cmd_msg.header.stamp = rclpy.clock.Clock().now().to_msg()
    cmd_msg.velocity =vels 
    cmd_pub.publish(cmd_msg)

def goto_xyzrpy(target, delay=0.1, tol=0.05, max_step=100, gain=1.0, stop=True):
    ee6=get_ee(ros_interface.latest_joint_states)
    xd=ee6-target
    xd[3:6]=0 

    # print('xd: ', xd)

    alpha = .0001
    I = np.eye(q.shape[0])

    step=0
    while np.linalg.norm(xd)>tol:
        ee6=get_ee(ros_interface.latest_joint_states)
        xd=target-ee6 
        xd = xd * gain
        xd[3:6]=0        # only xyz, rpy not working yet

        J = robot.getJacobian('camera_link')
        
        qd = np.linalg.inv(J.T @ J + alpha * I) @ J.T @ xd

        set_vels(list(qd))
        time.sleep(delay)

        step = step+1 
        if step>=max_step:
            break 
    
    # stop
    if stop:
        set_vels([0.0]*10)

apos=0
def goto_q_pos_velocity_ctrl(target_q, alpha=0.7, tol=0.01,  delay=0.1, maxstep=50):
    global apos
    target_jointpos=target_q
    cpos=robot.getJoints()
    dpos=np.linalg.norm(target_jointpos-cpos)

    step =0
    while dpos>tol: 
        cpos=robot.getJoints()
        dpos=np.linalg.norm(target_jointpos-cpos)
          

        qd =(target_jointpos-cpos)*alpha
        apos=qd 
        if np.max(qd) <0.001:
            print('reached')
            break 


        set_vels(list(qd))
        time.sleep(delay)
        step +=1

        if step>=maxstep:
            break 
        
    set_vels([0.0]*10)

### Robot safety check

In [11]:
pos1=robot.getJoints()
pos1

array([ 0.21210059,  0.05880957,  0.30893555, -2.12995605,  1.26585547,
       -1.06628027, -1.98705078,  2.39380273,  0.04069973,  0.        ])

### move little by hand, then execute the following

In [10]:
goto_q_pos_velocity_ctrl(pos1, tol=0.005)

### spoon pick

In [12]:
ee_init_pos_spoon=np.array([ 0.21559961, -0.50414063,  0.31746387, -2.12591309,  1.28041895,
       -1.06813965, -1.98603027,  2.38494531,  0.04077414,  0.        ])

In [13]:
goto_q_pos_velocity_ctrl(ee_init_pos_spoon, tol=0.005)

reached


'/home/carl/data_sawyer/spoon_hang/spoon_hang_10d_group.hdf5'

### Trajectory playing test

In [14]:
dataset_path =  config.train.data 
print(f'dataset_path: {dataset_path}')

f = h5py.File(dataset_path, "r")
demos = list(f["data"].keys())
len(demos)

dataset_path: /home/carl/data_sawyer/spoon_pick/expert/spoon_pick_100_10d.hdf5


100

In [15]:
demo_name="demo_1"
demo=f['data'][demo_name]
actions=demo['actions']
ee_pos=demo['obs']['robot0_eef_pos']
ee_pos.shape

(71, 6)

In [16]:
f['data'][demo_name]['obs'].keys()

<KeysViewHDF5 ['agentview_image', 'robot0_eef_pos', 'robot0_eye_in_hand_image', 'robot0_joint_pos', 'robot0_joint_vel']>

In [17]:
goto_xyzrpy(ee_pos[0], tol=0.005) 

In [17]:
for i in range(len(ee_pos)):
    goto_xyzrpy(ee_pos[i], delay=0.001, tol=0.02) 

goto_xyzrpy(ee_pos[i], tol=0.005)  # last one with precision

In [18]:
goto_xyzrpy(ee_pos[-1], tol=0.005)

In [20]:
eepos_current=ee_pos[0]
eepos_current

array([ 0.58733696, -0.03160624,  0.3367485 , -2.64198041,  1.55696198,
        2.13278455])

### Robot is ready

In [18]:
policy.start_episode()

In [19]:
def get_current_obs():
    # img_wrist=cam_wrist.get_current_frame(scale=0.5)  
    # img_front=cam_front.get_current_frame(scale=0.5) 

    image_wrist=ros_interface.image_wrist
    image_front=ros_interface.image_front

    if image_front.sum()==0.0:
        print('image front None')
        return None
    if image_wrist.sum()==0:
        print('image wrist None')
        return None 

    scale=0.5
    img_wrist = cv2.resize(image_wrist, (int(image_wrist.shape[1]*scale), int(image_wrist.shape[0]*scale) ) , interpolation = cv2.INTER_AREA)
    img_front = cv2.resize(image_front, (int(image_front.shape[1]*scale), int(image_front.shape[0]*scale) ) , interpolation = cv2.INTER_AREA)


    # print('shape: ', image_wrist.shape, image_front.shape)
    msg=ros_interface.latest_joint_states
    gripper_isopen=ros_interface.latest_gripper_state

    ee=get_ee(msg)

    try:
        pos =np.array([msg.position[ind] for ind in joint_inds])
        vel =np.array([msg.velocity[ind] for ind in joint_inds])
        t=msg.header.stamp.sec+msg.header.stamp.nanosec*1e-9
    except Exception as e:
        # print('msg=', msg)
        # print('Error=',e, e.st)
        # traceback.print_exc()
        return None 

    img_wrist=cv2.cvtColor(img_wrist, cv2.COLOR_BGR2RGB)
    img_front=cv2.cvtColor(img_front, cv2.COLOR_BGR2RGB)

    img_wrist=cv2.resize(img_wrist, (84,84))
    img_front=cv2.resize(img_front, (84,84))


    ob={'agentview_image': img_front, 'robot0_eye_in_hand_image': img_wrist, 'robot0_joint_pos': pos, 'robot0_joint_vel':vel, 'robot0_eef_pos': ee}
    ob['robot0_eye_in_hand_image']=ob['robot0_eye_in_hand_image'].transpose(2,0,1)
    ob['agentview_image']=ob['agentview_image'].transpose(2,0,1)

    return ob

In [20]:
ob=get_current_obs()
ob!=None 

True

In [21]:
ob.keys()

dict_keys(['agentview_image', 'robot0_eye_in_hand_image', 'robot0_joint_pos', 'robot0_joint_vel', 'robot0_eef_pos'])

In [58]:
goto_xyzrpy(ee_pos[0], tol=0.005) 

In [23]:
eepos_start=ee_pos[0]
eepos_current=eepos_start
eepos_current

array([ 0.58733696, -0.03160624,  0.3367485 , -2.64198041,  1.55696198,
        2.13278455])

In [64]:
cv2.destroyAllWindows() 

In [63]:
goto_q_pos_velocity_ctrl(ee_init_pos_spoon, tol=0.005)

### try

In [62]:
frequency = 20.0   # similar to robomimic


eepos_start=ee_pos[0]
eepos_current=eepos_start


st=time.time()
acs=[]
for i in range(1000000):
    ob=get_current_obs()
    if ob==None:
        continue

    ob2 = policy._prepare_observation(ob)
    ac = policy.policy.get_action(obs_dict=ob2, goal_dict=None)
    ac=ac.detach().cpu().numpy()[0]
    print(f'i={i} ac={ac}')
    acs.append(ac)

    if ac[-1]>0.9:
        print('---------------------------------')
        print('grip now-----------------------')
        print('\n---------------------------------------')
        tb=Thread(target=beep_start)
        tb.start()
        # break 

    eepos_current[:3] += ac[:3]
    print(f'eepos_togo: {eepos_current}')

    goto_xyzrpy(eepos_current, tol=0.01, max_step=2, stop=False)
    


    image_front=ob['agentview_image'].transpose(2,1,0) 
    image_wrist=ob['robot0_eye_in_hand_image'].transpose(2,1,0)
    

    image=np.concatenate([image_wrist, image_front], axis=1) 
    image= cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    cv2.imshow('frame',image)
    if cv2.waitKey(1) == 27: 
        print('UI closed')
        break  # esc to quit

    time.sleep(1.0/frequency)

cv2.destroyAllWindows() 

i=0 ac=[ 3.4744674e-04  8.5275259e-04  3.6695600e-04 -1.4438295e-01
 -2.6315782e-04 -1.4274940e-01  5.9720041e-05]
eepos_togo: [ 0.5876844  -0.03075349  0.33711546 -2.64198041  1.55696198  2.13278455]
i=1 ac=[ 0.0002559   0.00115734  0.00023348 -0.1146822  -0.00018388 -0.11377054
  0.00023219]
eepos_togo: [ 0.5879403  -0.02959615  0.33734893 -2.64198041  1.55696198  2.13278455]
i=2 ac=[ 4.7390588e-04  1.1996259e-03  5.2580697e-04 -9.8709226e-02
 -6.8104418e-07 -9.7469911e-02  3.2582897e-04]
eepos_togo: [ 0.58841421 -0.02839653  0.33787474 -2.64198041  1.55696198  2.13278455]
i=3 ac=[ 0.0004452   0.00123139  0.00035937 -0.09006767 -0.00020926 -0.08881456
  0.00016376]
eepos_togo: [ 0.5888594  -0.02716513  0.33823411 -2.64198041  1.55696198  2.13278455]
i=4 ac=[ 2.7037619e-04  1.0747578e-03  3.1013208e-04 -8.6700909e-02
 -3.9149141e-05 -8.5400529e-02  4.9724581e-04]
eepos_togo: [ 0.58912978 -0.02609037  0.33854425 -2.64198041  1.55696198  2.13278455]
i=5 ac=[ 2.3190462e-04  1.1044440e-03

ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred


i=48 ac=[ 5.30004385e-04  1.67454744e-03 -1.55587899e-04 -1.30950287e-01
 -1.15590825e-04 -1.29300430e-01  4.02606762e-04]
eepos_togo: [ 0.60909996  0.03080551  0.33852744 -2.64198041  1.55696198  2.13278455]
i=50 ac=[ 5.4056844e-04  1.7738190e-03 -5.5459113e-04 -1.2734379e-01
 -1.2512822e-04 -1.2568548e-01  3.4829718e-04]
eepos_togo: [ 0.60964053  0.03257933  0.33797285 -2.64198041  1.55696198  2.13278455]
i=51 ac=[ 0.00064171  0.00170706 -0.00018252 -0.13400982 -0.00019247 -0.13230482
  0.00020403]
eepos_togo: [ 0.61028224  0.03428638  0.33779033 -2.64198041  1.55696198  2.13278455]
i=52 ac=[ 0.0004778   0.00170919 -0.00034999 -0.12574892 -0.00030357 -0.12409291
  0.00047431]
eepos_togo: [ 0.61076004  0.03599558  0.33744034 -2.64198041  1.55696198  2.13278455]
i=53 ac=[ 5.4596231e-04  1.5398565e-03  1.3120912e-04 -1.6696894e-01
 -4.5437546e-04 -1.6566703e-01  1.4487165e-04]
eepos_togo: [ 0.611306    0.03753543  0.33757155 -2.64198041  1.55696198  2.13278455]
i=54 ac=[ 5.8427110e-04  

ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred


i=59 ac=[ 0.00094522  0.00179489 -0.00036263 -0.17215694 -0.0003468  -0.17025268
  0.00028446]
eepos_togo: [ 0.61593316  0.04773115  0.33588992 -2.64198041  1.55696198  2.13278455]
i=60 ac=[ 0.00064014  0.00190561 -0.00057566 -0.16082823 -0.00032242 -0.15885946
  0.00042157]
eepos_togo: [ 0.61657329  0.04963675  0.33531426 -2.64198041  1.55696198  2.13278455]
i=61 ac=[ 3.9078202e-04  1.6324214e-03 -1.4850275e-04 -1.4031225e-01
 -7.5351425e-05 -1.3848478e-01  2.3933436e-04]
eepos_togo: [ 0.61696407  0.05126917  0.33516576 -2.64198041  1.55696198  2.13278455]
i=62 ac=[ 5.3392351e-04  1.2071652e-03 -9.7331111e-05 -5.8814473e-02
 -7.3769217e-05 -6.0036406e-02  1.0000963e+00]
---------------------------------
grip now-----------------------

---------------------------------------
eepos_togo: [ 0.617498    0.05247634  0.33506843 -2.64198041  1.55696198  2.13278455]
i=63 ac=[ 3.28978989e-04  1.55514095e-03  1.01896556e-04 -1.56512246e-01
 -4.09096188e-04 -1.55660421e-01  3.20509425e-04]
eepo

ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred


i=66 ac=[ 0.00014835  0.00169102 -0.00032907 -0.11882201 -0.00037863 -0.11765336
  0.00028148]
eepos_togo: [ 0.61881132  0.05668718  0.33434669 -2.64198041  1.55696198  2.13278455]
i=67 ac=[ 4.31289227e-04  1.57424912e-03 -2.72714940e-04 -1.19769424e-01
  3.76565731e-05 -1.18514664e-01  3.44153465e-04]
eepos_togo: [ 0.61924261  0.05826143  0.33407398 -2.64198041  1.55696198  2.13278455]
i=69 ac=[ 0.00019329  0.00164797 -0.00044271 -0.11433533 -0.00014952 -0.1128673
  0.00030754]
eepos_togo: [ 0.6194359   0.0599094   0.33363127 -2.64198041  1.55696198  2.13278455]
i=70 ac=[ 3.9650081e-04  1.6223829e-03 -3.2792758e-04 -1.2047234e-01
 -4.8438975e-05 -1.1887283e-01  1.6214125e-04]
eepos_togo: [ 0.6198324   0.06153178  0.33330334 -2.64198041  1.55696198  2.13278455]
i=71 ac=[ 0.00023592  0.00174894 -0.00048911 -0.10853326 -0.00024717 -0.10725672
  0.00013792]
eepos_togo: [ 0.62006832  0.06328073  0.33281423 -2.64198041  1.55696198  2.13278455]
i=72 ac=[ 0.00010924  0.00183639 -0.00044571 -0

KeyboardInterrupt: 

In [38]:
goto_xyzrpy(ee_pos[0], tol=0.005)  

In [29]:
ac

array([-3.6969720e-04,  1.2310240e-03,  3.0301284e-04, -1.6575159e-03,
        4.3890024e-05, -1.8973439e-03, -2.0936792e-04], dtype=float32)

In [42]:
cv2.destroyAllWindows() 

### success: worked for the first time: (note: doesn't stop)

In [51]:
# eepos_current[:3] += ac 
# eepos_current

array([ 0.54482866,  0.15848914,  0.34229148, -0.0208837 ,  0.75747543,
       -1.54219383])

In [ ]:
goto_xyzrpy(ee_pos[0], tol=0.005) 

In [31]:
cv2.destroyAllWindows() 

In [36]:
acs2=np.vstack([acs])
acs2.shape

(57, 7)

In [37]:
np.sum(acs2[:,6])

1.0015614

In [38]:
acs2[:,6] > 0.8

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False])

In [37]:

np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

In [38]:
acs2[:,6] 

array([-0.,  0., -0., -0., -0.,  0., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,
       -0., -0., -0., -0., -0., -0.,  0., -0., -0., -0.,  0., -0., -0.,
       -0., -0., -0.,  0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.],
      dtype=float32)

In [34]:
for i,ac in enumerate(acs2):
    if ac[-1]>0.9:
        print(i, 'grip now')

In [35]:
ac[-1]

-0.00024590347